# MAC0219 - Programação Concorrente, Paralela e Distribuída
# EP2: CUDA & OpenMPI


| Nome | NUSP |
|------|------|
| Caio Andrade | 9797232 |
| Caio Fontes | 10692061 |
| Eduardo Laurentino | 8988212 |
| Thiago Teixeira | 10736987 |
| Washington Meireles | 10737157 |

Neste relatório, vamos explicar...

Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum.

In [1]:
using DataFrames, Query, StatsPlots, Statistics, CSV

In [2]:
function parse_results(results, par)
    if par == "processes"
        parsed_results = results |>
                        @groupby({_.processes,}) |>
                        @map({processes = key(_).processes,
                              mean_duration = mean(_.duration),
                              ci_duration = 1.96 * std(_.duration)}) |>
                        DataFrame
    else
            parsed_results = results |>
                        @groupby({_.dimensions,}) |>
                        @map({dimensions = key(_).dimensions,
                              mean_duration = mean(_.duration),
                              ci_duration = 1.96 * std(_.duration)}) |>
                        DataFrame
        end

    return parsed_results
end

parse_results (generic function with 1 method)

In [3]:
function save_csv_results(parsed_results, name)
    CSV.write(string(name, ".csv"), parsed_results)
end

save_csv_results (generic function with 1 method)

In [4]:
function read_csv_results(filename)
    return CSV.read(filename)
end

read_csv_results (generic function with 1 method)

# 1. Implementação em CUDA

A implementação em CUDA foi feita tendo como base a versão sequencial, a qual adaptamos para as necessidades da paralelização com computação heterogênea, como é o caso aqui. Nesse sentido, transformamos o método `void compute_mandelbrot()` num método global, pois é este o **Kernel** do programa - isto é, o programa que conecta o _Host_ (CPU) com o _Device_ (GPU). Além disso, deixamos os métodos que são utilizados somente durante o processamento dos pixels (à saber, `int mandelbrot()` e `void update_rgb_buffer()` como exclusivas para o _device_. O restante das conexões entre as duas camadas de memória, do _Host_ e do _Device_ , foi feito manipulando adequadamente a alocação e tráfego de memória quando necessário. 

A divisão de tarefas é feita de maneira estática à partir das dimensões dos _Blocos_ e do _Grid_ no qual estes estão inseridos. Para que pudessemos determinar experimentalmente os melhores valores para essas dimensões, o programa solicita como argumento (além das coordenadas da região do conjunto e o tamanho da imagem) as dimensões $(x, y)$ dos blocos. A partir desses valores, determina-se também as dimensões do _Grid_ de forma a garantir que a divisão das tarefas contemple toda a imagem a ser gerada. No caso da nossa implementação, isso significa atribuir ao _Grid_ dimensões $(\frac{image\_size}{x}, \frac{image\_size}{y})$. 

Dessa forma, fica estabelecido que cada pixel $(i_x, i_y)$ da imagem será tratado por uma _thread_ de identidade determinada pela sua posição num determinado bloco do grid de acordo com as especificações abaixo:

$$
i_x = blockIdx.x*blockDim.x+threadIdx.x \\
i_y = blockIdx.y*blockDim.y+threadIdx.y
$$

É importante observar uma condição de execução importante relativa a essa implementação: devemos ter, necessariamente, $x * y \leq 1024$, pois ao determinarmos as dimensões dos blocos como sendo $(x, y)$, isso significa que a quantidade de _threads_ em cada bloco é igual a $x * y$.O limite de $1024$ _threads_ por bloco é uma condição própria do desenvolvimento em CUDA.

Compilação:

In [5]:
;make mandelbrot_cuda

nvcc -o mandelbrot_cuda -gencode arch=compute_50,code=[sm_50,compute_50] -Wno-deprecated-gpu-targets src/mandelbrot_cuda.cu


/bin/sh: nvcc: command not found
make: *** [mandelbrot_cuda] Error 127


## 1.1 Experimentos com CUDA

Seguindo as instruções do enunciado, os experimentos foram feitos usando os seguintes parâmetros fixos: $15$ **repetições** da geração de uma imagem de **tamanho** $4096$ da região **Triple Spiral Valley**.

Posto isso, o objetivo aqui é realizar experimentos para diferentes valores dos parâmetros $x$ e $y$, que determinam a dimensão dos blocos e do _grid_. Para tanto, determinamos uma região de interesse para esses parâmetros cujo principal critério de escolha foi a limitação de mantermos $x * y \leq 1024$ conforme explicado anteriormente. Nesse sentido, decidimos começar com blocos de dimensao unica (isto é, uma única thread por bloco, implicando num grid com $image\_size * image\_size$ blocos) e irmos dobrando até $(32, 32)$, onde têm-se em cada bloco o limite de $1024$ threads e com _grid_ de dimensão $(128, 128)$. 

A **região de interesse** final, com a qual realizamos os experimentos, é:

**Dimensões dos blocos:** $(1,1)$, $(2,2)$, $(4,4)$, $(8,8)$, $(16,16)$ e $(32,32)$ 

**Respectivas dimensoes do _grid_ :** $(4096, 4096)$, $(2048, 2048)$, $(1024, 1024)$, $(512, 512)$, $(256, 256)$ e $(128, 128)$ 

In [6]:
dimensions = [(2,2),(4,4),(8,8),(16,16),(32,32)]

5-element Array{Tuple{Int64,Int64},1}:
 (2, 2)
 (4, 4)
 (8, 8)
 (16, 16)
 (32, 32)

In [7]:
function mandelbrot_cuda(size, x, y)
    """ Executa o cálculo do conjunto de Mandelbrot na região TripleSpiralValley
        gerando uma imagem de tamanho _size_ fazendo uso da implementação
        em CUDA com blocos de dimensao _(x, y)_ e grid de dimensao _(size/x, size/y)_
    """
    time = parse.(Float64, chomp(read(`./mandelbrot -0.188 -0.012 0.554 0.754 $size $x $y`, String)))
    return DataFrame(dimensions = string(x, ", ", y), duration = time)
end

mandelbrot_cuda (generic function with 1 method)

In [8]:
function run_experiments_cuda(dimensions, size=4096, repetitions=15)
    """ Obtem a duracao dos tempos de execução associado aos cálculos de
        conjunto de Mandelbrot na região TripleSpiralValley à partir da versão 
        paralelizada em CUDA para imagens de tamanho _size_, com uma quantidade
        igual a _repetitions_ de repetições para cada uma das dimensoes de bloco
        (e, por consequencia, de grid) presentes em _dimensions_
    """
    
    results = DataFrame(dimensions = String[], duration = Float64[])    
    
    for x_y in dimensions
        x, y = x_y
        for i in 1:repetitions
            append!(results, mandelbrot_cuda(size, x, y))    
        end
    end
    
    return results
end

run_experiments_cuda (generic function with 3 methods)

In [41]:
results_cuda = run_experiments_cuda(dimensions, 512, 15)
#save_csv_results(results_cuda, "cuda_experiments")

Base.IOError: IOError: could not spawn `./mandelbrot -0.188 -0.012 0.554 0.754 512 2 2`: no such file or directory (ENOENT)

In [43]:
save_csv_results(results_cuda, "cuda_experiments")
experiments_cuda = read_csv_results("cuda_experiments.csv")
final_results_cuda = parse_results(experiments_cuda, "dimensions")

┌ Warning: `CSV.read(input; kw...)` is deprecated in favor of `DataFrame!(CSV.File(input; kw...))`
└ @ CSV /Users/caioandrade/.julia/packages/CSV/W9RT2/src/CSV.jl:40


,dimensions,mean_duration,ci_duration
,String,Float64,Float64
1,"2, 2",0.0248423,0.00222969
2,"4, 4",0.0250052,0.00295183
3,"8, 8",0.0248343,0.00108345
4,"16, 16",0.0244791,0.00111605
5,"32, 32",0.0243553,0.000819009


# 2. Implementação em OMPI

A implementação em OMPI também teve como base a implementação sequencial do programa, que foi adequada para a paralelização através da troca de mensagens. Para este fim, a função `compute_mandelbrot` foi adaptada para receber parâmetros adicionais e uma função responsável por gerenciar a troca de mensagens - `compute_mandelbrot_ompi`- foi criada. Iremos explicar a implementação descrevendo o que é feito pelo processo principal (de rank 0, também chamado de `MASTER` no código) e pelos processos auxiliares.

**Processo Principal:**

No processo principal, primeiramente são inicializadas várias variáveis globais e os tipos de dados a serem utilizados pelas mensagens são definidos. Logo a seguir, o processo realiza a inicialização e os cálculos dos parametros a serem passados a cada outro processo para definir em que região da imagem ele irá trabalhar. 

A imagem é dividida em quadrantes com lado de tamanho $(image\_size\div\sqrt{n})+ 1$ arredondado para um número inteiro, onde $n$ é o número de processos que realizam trabalho, o último processo sempre se torna responsável por toda a parte restante quando chega o momento de definir o seu trabalho, em situações normais isso equivale a apenas mais um quadrante. Se toda a imagem já tiver sido atribuída e ainda tivermos processos restantes eles não realizam nenhuma computação. O intuito dessa divisão é garantir que toda a imagem seja coberta com qualquer número de processos e para qualquer tamanho de imagem, mesmo que certa ineficiências ocorram na divisão.

O processo principal então manda mensagens para todos os processos auxiliares com os valores que definem as coordenadas de começo e término da região que cada um deverá processar. Em seguida ele espera os processos terminarem a computação.

O processo principal então recebe um vetor 1D com os valores de $x$,$y$ e $iterations$ para cada pixel, e uma variável `count` indicando o tamanho desse vetor. Depois de receber os resultados de todos processos o processo principal transfere os valores para a imagem determinado as cores correspondentes.

**Processos Auxiliares**

Os processos auxiliares realizam o mesmo processo de inicialização das variáveis globais e dos tipos de dados. Depois disso esperam, através da chamada `MPI_Recv` uma mensagem do processo principal definindo a sua região da imagem.

Em outras palavras, fizemos uma divisão de trabalho entre processo principal e processos auxiliares de forma que o processo principal ficou encarregado de **gerenciar** o trabalho que cada processo auxiliar executa, ou seja, ele é encarregado de fazer a divisão de trabalho, inicializar variáveis e orquestrar o trabalho entre os processos auxiliares, **enviando** trechos da imagem que cada processo deverá calcular e **recebendo** o resultado, e consequentemente montando a imagem resultado. Os processos auxiliares simplesmente calculam como sua região da imagem deve ser colorida e enviam para o processo principal.

Compilação:

In [11]:
;make mandelbrot_ompi

mpicc -o mandelbrot_ompi -lm src/mandelbrot_ompi.c


make: mpicc: No such file or directory
make: *** [mandelbrot_ompi] Error 1


## 2.1 Experimentos com OMPI

Seguindo as instruções do enunciado, os experimentos foram feitos usando os seguintes parâmetros fixos: $15$ **repetições** da geração de uma imagem de **tamanho** $4096$ da região **Triple Spiral Valley**.

O objetivo é determinar um número ideal $n$ de processos para a implementação através desses experimentos. Definimos a região de interesse conforme sugestão do enunciado, mas sempre tendo pelo menos um processo principal que coletasse os resultados, sendo assim a **região de interesse** com que realizamos os experimentos foi:

**Número de Processos:** $2$,$3$,$5$,$9$,$17$,$33$,$65$


In [12]:
#definir regiao
processes = [2,3,5,9,17,33,65]

7-element Array{Int64,1}:
  2
  3
  5
  9
 17
 33
 65

In [13]:
#funcao pra rodar experimentos nessa regiao
function mandelbrot_ompi(size,n_proc)
    """ Executa o cálculo do conjunto de Mandelbrot na região TripleSpiralValley
        gerando uma imagem de tamanho _size_ fazendo uso da implementação
        em OMPI com _n_proc_ processos.
    """
    time = parse.(Float64, chomp(read(`mpirun -np $n_proc --host localhost:$n_proc mandelbrot_ompi -0.188 -0.012 0.554 0.754 $size`, String)))
    return DataFrame(processes = string(n_proc), duration = time)
end

mandelbrot_ompi (generic function with 1 method)

In [14]:
#funcao pra rodar experimentos nessa regiao
function run_experiments_ompi(process_range, size=4096, repetitions=15)
    """ Obtem a duracao dos tempos de execução associado aos cálculos de
        conjunto de Mandelbrot na região TripleSpiralValley à partir da versão 
        paralelizada em OMPI para imagens de tamanho _size_, com uma quantidade
        igual a _repetitions_ de repetições para cada número de processos em
        _process_range_
    """
    
    results = DataFrame(processes = String[], duration = Float64[])    
    
    for n in process_range
        for i in 1:repetitions
            append!(results, mandelbrot_ompi(size,n))    
        end
    end
    
    return results
end

run_experiments_ompi (generic function with 3 methods)

In [46]:
#rodar os experimentos e salvar em csv

results_ompi = run_experiments_ompi(processes) 
save_csv_results(results_ompi, "ompi_experiments")

Base.IOError: IOError: could not spawn `mpirun -np 2 --host localhost:2 mandelbrot_ompi -0.188 -0.012 0.554 0.754 4096`: no such file or directory (ENOENT)

In [16]:
#ler o csv e gerar o parser com medias e CIs
experiments_ompi = read_csv_results("ompi_experiments.csv")

# Não sei qual erro está dando
final_results_ompi = parse_results(experiments_ompi, "processes") 

┌ Warning: `CSV.read(input; kw...)` is deprecated in favor of `DataFrame!(CSV.File(input; kw...))`
└ @ CSV /Users/caioandrade/.julia/packages/CSV/W9RT2/src/CSV.jl:40


,processes,mean_duration,ci_duration
,Int64,Float64,Float64
1,2,26.7508,0.215736
2,3,26.7096,0.198727
3,5,9.23427,0.192052
4,9,9.45713,0.452817
5,17,4.68719,0.367088
6,33,4.10299,0.438417
7,65,4.25879,0.279031


# 3. Implementação em OMPI + OMP

Nessa implementação, mantivemos a estrutura de comunicação definida na implementação com OMP e alteramos como cada processo auxiliar processa sua região da imagem: para calcular o número de iterações até a convergência de cada pixel, paralelizamos a computação por OpenMP. Para tanto, as mudanças ocorreram apenas na função `compute_mandelbrot`.

Compilação:

In [17]:
;make mandelbrot_ompi_omp

mpicc -o mandelbrot_ompi_omp -lm -fopenmp src/mandelbrot_ompi_omp.c


make: mpicc: No such file or directory
make: *** [mandelbrot_ompi_omp] Error 1


## 3.1 Experimentos com OMPI + OMP

Seguindo as instruções do enunciado, os experimentos foram feitos usando os seguintes parâmetros fixos: $15$ **repetições** da geração de uma imagem de **tamanho** $4096$ da região **Triple Spiral Valley**.

Os números de processos foram determinados da mesma maneira que na implementação em OMPI, para fins de equivalência dos experimentos. O número $t$ de threads varia entre 1 e 64 seguindo as potências de 2, afim de cobrir uma região grande, permitindo analisar o impacto desse parâmetro. Consideramos que valores maiores gerariam um overhead muito grande, tornando o experimento pouco informativo.

**FALAR/JUSTITIFICAR DEFINIÇÃO DA REGIAO DE INTERESSE!**

**Número de Processos:** $2$,$3$,$5$,$9$,$17$,$33$,$65$

**Número de Threads:** $1$,$2$,$4$,$8$,$16$,$32$,$64$


In [18]:
#definir regiao
processes = [2,3,5,9,17,33,65]
threads = [1,2,4,8,16,32,64]

7-element Array{Int64,1}:
  1
  2
  4
  8
 16
 32
 64

In [19]:
#funcao pra rodar experimentos nessa regiao
function mandelbrot_ompi_omp(size,n_proc,n_threads)
    """ Executa o cálculo do conjunto de Mandelbrot na região TripleSpiralValley
        gerando uma imagem de tamanho _size_ fazendo uso da implementação
        em OMPI com _n_proc_ processos.
    """
    time = parse.(Float64, chomp(read(
                `mpirun -np $n_proc --host localhost:$n_proc mandelbrot_ompi_omp -0.188 -0.012 0.554 0.754 $size $n_threads`, 
                String)
            ))
    return DataFrame(processes = string(n_proc),threads=string(n_threads), duration = time)
end

mandelbrot_ompi_omp (generic function with 1 method)

In [20]:
#funcao pra rodar experimentos nessa regiao
function run_experiments_ompi_omp(process_range,thread_range, size=4096, repetitions=15)
      """ Obtem a duracao dos tempos de execução associado aos cálculos de
        conjunto de Mandelbrot na região TripleSpiralValley à partir da versão 
        paralelizada em OMPI e OMP para imagens de tamanho _size_, com uma quantidade
        igual a _repetitions_ de repetições para cada número de processos em
        _process_range_ e número de threads em _thread_range_.
    """
    
    results = DataFrame(processes = String[],threads=String[], duration = Float64[])    
    
    for n in process_range
        for t in thread_range
            for i in 1:repetitions
                append!(results, mandelbrot_ompi_omp(size,n,t))
            end
        end
    end
    
    return results
end

run_experiments_ompi_omp (generic function with 3 methods)

In [21]:
#rodar os experimentos e salvar em csv 
results_ompi_omp = run_experiments_ompi_omp(processes,threads,256,5)

Base.IOError: IOError: could not spawn `mpirun -np 2 --host localhost:2 mandelbrot_ompi_omp -0.188 -0.012 0.554 0.754 256 1`: no such file or directory (ENOENT)

In [22]:
#ler o csv e gerar o parser com medias e CIs
save_csv_results(results_ompi_omp, "ompi_omp_experiments")
experiments_ompi_omp = read_csv_results("ompi_omp_experiments.csv")
final_results_ompi_omp = parse_results(experiments_ompi_omp, "processes") 

UndefVarError: UndefVarError: results_ompi_omp not defined

# 4. Implementação em OMPI + CUDA

Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum.

Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum.

Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum.

Compilação:

## 4.1 Experimentos com OMPI + CUDA

Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum.

Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum.

**FALAR/JUSTITIFICAR DEFINIÇÃO DA REGIAO DE INTERESSE!**

In [23]:
#definir regiao

In [24]:
#funcao pra rodar experimentos nessa regiao

In [25]:
#funcao pra rodar experimentos nessa regiao

In [26]:
#rodar os experimentos e salvar em csv 

In [27]:
#ler o csv e gerar o parser com medias e CIs

# 5. Análise dos resultados dos experimentos

Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum.

Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum.

Faremos uso das duas funções abaixo para gerar os gráficos sobre os quais analisaremos cada caso:

In [28]:
function plot_results_scatter(x, xlabel_, ylabel_, title_,
                        y1, series_label, yerror1)  
    p = plot(x,        
            y1,
            yerror = yerror1,
            alpha = 0.9,
            labels = series_label,
            xlabel = xlabel_,
            ylabel = ylabel_,
            title = title_,
            color = "red",
            seriestype = :scatter,
            lw = 1,
            legend = :topright)
    return p
end

plot_results_scatter (generic function with 1 method)

In [29]:
#implementar função pra gear grafico de todos os pontos
function plot_results(x, y, series_label, xlabel_, ylabel_, title_)      
    p = scatter(x, y,
            alpha = 0.6,
            labels = series_label,
            xlabel = xlabel_,
            ylabel = ylabel_,
            title = title_,
            legend = :topright)
    return p
end



plot_results (generic function with 1 method)

## 5.1 Resultados da versão CUDA

[ANALISE GERAL]

Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum.

Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum.


In [30]:
#plotar graficos com todos os pontos


plot_results(results_cuda.dimensions,
    results_cuda.duration,    
    "Duração", "Dimensão", "Tempo(s)", "Experimentos em CUDA")



UndefVarError: UndefVarError: results_cuda not defined

In [31]:
plot_results_scatter(final_results_cuda.dimensions, "Dimensoes", "Tempo(s)", "CUDA",
                     final_results_cuda.mean_duration, "Tempo médio de execução", 
                    final_results_cuda.ci_duration)

UndefVarError: UndefVarError: final_results_cuda not defined

**ESCOLHER E JUSTIFICAR OS MELHORES PARAMETROS!**

Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum.

Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum.

## 5.2 Resultados da versão OMPI

[ANALISE GERAL]

Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum.

Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum.


In [32]:
#plotar graficos com todos os pontos

In [33]:
#plotar graficos com tempo medio e intervalo de confiança

**ESCOLHER E JUSTIFICAR OS MELHORES PARAMETROS BASEANDO-SE NOS GRAFICOS!**

Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum.

Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum.

## 5.3 Resultados da versão OMPI + OMP

[ANALISE GERAL]

Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum.

Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum.


In [34]:
#plotar graficos com todos os pontos

In [35]:
#plotar graficos com tempo medio e intervalo de confiança

**ESCOLHER E JUSTIFICAR OS MELHORES PARAMETROS BASEANDO-SE NOS GRAFICOS!**

Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum.

Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum.

## 5.4 Resultados da versão OMPI + CUDA

[ANALISE GERAL]

Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum.

Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum.


In [36]:
#plotar graficos com todos os pontos

In [37]:
#plotar graficos com tempo medio e intervalo de confiança

**ESCOLHER E JUSTIFICAR OS MELHORES PARAMETROS BASEANDO-SE NOS GRAFICOS!**

Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum.

Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum.

# 6. Comparação entre as diferentes versões

Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum.

Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum.

In [38]:
#plotar graficos!!!

In [39]:
#plotar graficos!!!

In [40]:
#plotar graficos!!!

***Usando os parâmetros determinados na seção anterior, comparar os desempenhos das seguintes versões do programa basenado-se nos graficos!!!!***

Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum.

Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum.

Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum.

Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum.

# 7. Conclusão

Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum.

Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum.

Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum.

Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum.